In [1]:
import pandas as pd
import numpy as np
import itertools
from sklearn.cluster import KMeans

In [2]:
PATH = "./data/Sales_Transactions_Dataset_Weekly.csv"
DUMMY_COLUMNS = [
    "Product_Code"
]

In [3]:
# metric functions
# --------------------------------------------
def total_sum_of_squares_df(df, centroid = None):
    """ Calculates and returns the TTS of the given DataFrame """
    if centroid is None:
        centroid = find_centroid_df(df)
        
    return total_sum_of_squares(df.as_matrix(), centroid)

def total_sum_of_squares(data, centroid):
    """ Calculates and returns the TTS of the given matrix
    
    Arguments:
      data - Iterable<Iterable>
      centroid - Array
    """        
    total = 0
    
    for row in data:
        for index, value in enumerate(row):
            diff = value - centroid[index]
            diffsq = diff ** 2
            total += diffsq
            
    return total


def find_centroid_df(df):
    """ Calculates and returns the centroid for a DataFrame """
    return df.mean()

# clustering functions
# --------------------------

def get_cluster_indexes(cluster_assignments):
    cluster_slices = {}
    
    for index, assignment in enumerate(assignments):
        if assignment not in cluster_slices:
            cluster_slices[assignment] = list()
            
        cluster_slices[assignment].append(index)
        
    return cluster_slices

def get_cluster_data(df, assignments):
    cluster_indexes = get_cluster_indexes(assignments)
    
    cluster_data = {k: df.iloc[v] for k, v in cluster_indexes.items()}
    
    return cluster_data

def get_clusters(df, assignments):
    """Returns an array of tuples with (<cluster>, <cluster_centroid>, <cluster_data_points>)"""
    
    return [
        (cluster, find_centroid_df(cluster_data), cluster_data) 
        for cluster, cluster_data 
        in get_cluster_data(df, assignments).items()
    ]
    

# model runners
# ------------------------------------------

#def run_gaussian_mixture(model, data):
    model.fit(data)
    return model.predict(data)

def run_kmeans(model, data):
    model.fit(data)
    return model.predict(data)

#def run_hclustering(model, data):
    return model.fit_predict(data)

# data functions 
# -------------------------------------------

def clean_data(data):
    # Strip whitespaces from all string values
    # and replace "?" with None,
    # and drop all na rows
    data = data.apply(lambda x: x.str.strip() if x.dtype == "object" else x) \
               .replace(["?"], [None]) \
               .dropna()

    return data

def prepare_data(data):
    return pd.get_dummies(data, columns=DUMMY_COLUMNS)

def read_data(path):
    dataset = pd.read_csv(path, header=0)
    dataset = clean_data(dataset)
    dataset = prepare_data(dataset)
    return dataset

In [4]:
df = read_data(PATH)

In [5]:
tss = total_sum_of_squares_df(df)
print("tss for data = %s" % tss)

tss for data = 6314001.08426


In [6]:
models = [
    ("KMeans", lambda k: KMeans(n_clusters=k), run_kmeans)
    ]

In [7]:
for model_name, create_model, run_model in models:
    print("-------------------------------")
    print(model_name)
    print("-------------------------------")
    print("")
    for k in range(2,10):
        print("Calculating %s clusters..." % k)
        print("")
        model = create_model(k)
        assignments = run_model(model, df)
        clusters = get_clusters(df, assignments)

        twss = 0
        for cluster, centroid, cluster_slice in clusters:
            cluster_tss = total_sum_of_squares_df(cluster_slice, centroid)
            print("cluster %s | tss = %s | size = %s" % (cluster, cluster_tss, len(cluster_slice)))
            twss += cluster_tss

        print("twss/tss = %s/%s = %s" % (twss, tss, twss / tss))
        print("")

-------------------------------
KMeans
-------------------------------

Calculating 2 clusters...

cluster 0 | tss = 905973.114504 | size = 667
cluster 1 | tss = 545136.779334 | size = 144
twss/tss = 1451109.89384/6314001.08426 = 0.22982414391

Calculating 3 clusters...

cluster 2 | tss = 267592.717997 | size = 197
cluster 0 | tss = 134070.912504 | size = 490
cluster 1 | tss = 308460.928885 | size = 124
twss/tss = 710124.559386/6314001.08426 = 0.112468235262

Calculating 4 clusters...

cluster 3 | tss = 119579.584783 | size = 164
cluster 0 | tss = 122884.926345 | size = 482
cluster 2 | tss = 66633.9667111 | size = 45
cluster 1 | tss = 282654.440475 | size = 120
twss/tss = 591752.918314/6314001.08426 = 0.0937207501896

Calculating 5 clusters...

cluster 2 | tss = 110425.82482 | size = 156
cluster 0 | tss = 50329.3856612 | size = 188
cluster 3 | tss = 66633.9667111 | size = 45
cluster 1 | tss = 282654.440475 | size = 120
cluster 4 | tss = 11730.1060262 | size = 302
twss/tss = 521773.7236